In [1]:
import pandas as pd
from dw import pg
import logging
import random
logging.basicConfig(
    level = logging.INFO
    ,format = "%(asctime)s [%(levelname)s] %(message)s")

In [2]:
n = 10000
df = pd.DataFrame(
    {
        "id": range(n)
        ,"score": [random.random() for i in range(n)]
        ,"amt": [random.choice(range(1000)) for i in range(n)]
        ,"cat": [random.choice(["test", "train"]) for i in range(n)]
        ,'time': [
                random.choice(
                    [pd.Timestamp(i) for i in 
                        ["20130102","20130202","20130302"]]
                ) 
                for i in range(n)]
    }
)
df.head()

,id,score,amt,cat,time
0,0,0.764839,195,test,2013-03-02
1,1,0.897606,618,train,2013-01-02
2,2,0.853854,609,train,2013-02-02
3,3,0.747254,320,test,2013-01-02
4,4,0.195149,698,test,2013-03-02


In [3]:
pg.drop('test')
pg.create(
        tbl_nme = 'test'
        ,dtypes = {
            'id':'integer'
            ,'score':'float8'
            ,'amt':'integer'
            ,'cat':'varchar(20)'
            ,'time':'timestamp'
            ,'constraint df_pk':
                'primary key (id)'
        }
    )
pg.write(df,'test')

2022-01-20 23:50:19,697 [INFO] running:
drop table test
2022-01-20 23:50:19,704 [INFO] done
2022-01-20 23:50:19,713 [INFO] running:
create table test(
    id integer
    ,score float8
    ,amt integer
    ,cat varchar(20)
    ,time timestamp
    ,constraint df_pk primary key (id)
)
2022-01-20 23:50:19,764 [INFO] done
2022-01-20 23:50:19,861 [INFO] running:
insert into test (id,score,amt,cat,time) values (:id,:score,:amt,:cat,:time)
2022-01-20 23:50:19,862 [INFO] len(args) = 10000
2022-01-20 23:50:21,643 [INFO] done


In [6]:
pg.qry('test').where("score > 0.5") \
.valc('time,cat',"avg(score) avgscore,round(sum(amt)/1e3,2) total") \
.pivot('time','cat',['n','avgscore','total'])

2022-01-20 23:52:44,145 [INFO] running:
with x as (
    select * from test
    where score > 0.5
)
select 
    time,cat
    ,count(1) n
    ,avg(score) avgscore,round(sum(amt)/1e3,2) total
from x
group by time,cat
order by n desc
2022-01-20 23:52:44,184 [INFO] done


n        avgscore             total        
cat        test train      test     train    test   train
time                                                     
2013-01-02  855   789  0.748191  0.750798  423.06  393.58
2013-02-02  828   829  0.747892  0.755623  402.04  415.65
2013-03-02  839   855  0.751682   0.75397  423.70  419.70

In [7]:
pg.run('select * from test limit 2')
pg.run('select * from test where score > :score limit 2'
    ,args = {'score':'0.9'})
pg.run('select * from test where score > :score limit 2'
    ,score = 0.9)

2022-01-20 23:56:29,624 [INFO] running:
select * from test limit 2
2022-01-20 23:56:29,626 [INFO] done
2022-01-20 23:56:29,628 [INFO] running:
select * from test where score > :score limit 2
2022-01-20 23:56:29,628 [INFO] len(args) = 1
2022-01-20 23:56:29,630 [INFO] done
2022-01-20 23:56:29,633 [INFO] running:
select * from test where score > 0.9 limit 2
2022-01-20 23:56:29,634 [INFO] done


,id,score,amt,cat,time
0,17,0.931387,573,train,2013-02-02
1,23,0.927040,309,test,2013-01-02


In [9]:
pg.qry('test').top()

2022-01-20 23:57:35,335 [INFO] running:
with x as (
    select * from test
)
select * from x limit 1
2022-01-20 23:57:35,337 [INFO] done


id                         0
score               0.764839
amt                      195
cat                     test
time     2013-03-02 00:00:00
Name: 0, dtype: object

In [10]:
pg.qry('test').head()

2022-01-20 23:57:35,352 [INFO] running:
with x as (
    select * from test
)
select * from x limit 5
2022-01-20 23:57:35,354 [INFO] done


,id,score,amt,cat,time
0,0,0.764839,195,test,2013-03-02
1,1,0.897606,618,train,2013-01-02
2,2,0.853854,609,train,2013-02-02
3,3,0.747254,320,test,2013-01-02
4,4,0.195149,698,test,2013-03-02


In [11]:
pg.qry('test').len()

2022-01-20 23:57:35,382 [INFO] running:
with x as (
    select * from test
)
select count(1) from x
2022-01-20 23:57:35,385 [INFO] done


10000

In [13]:
pg.qry('test').select("id,score,amt").top()
pg.qry('test').select(["id","score","amt"]).top()
pg.qry('test').select("id","score","amt").top()

2022-01-20 23:58:08,920 [INFO] running:
with x as (
    select id,score,amt from test
)
select * from x limit 1
2022-01-20 23:58:08,922 [INFO] done
2022-01-20 23:58:08,923 [INFO] running:
with x as (
    select id,score,amt from test
)
select * from x limit 1
2022-01-20 23:58:08,925 [INFO] done
2022-01-20 23:58:08,927 [INFO] running:
with x as (
    select id,score,amt from test
)
select * from x limit 1
2022-01-20 23:58:08,929 [INFO] done


id         0.000000
score      0.764839
amt      195.000000
Name: 0, dtype: float64

In [14]:
(
    pg.qry('test x')
    .select('x.id','y.id as yid','x.score','z.score as zscore')
    .join("test y","x.id = y.id+1","x.id <= y.id+1")
    .join("test z","x.id = z.id+2","x.id >= z.id+1")
    .where('x.id < 10','z.id < 10')
    .head()
)

2022-01-20 23:58:19,200 [INFO] running:
with x as (
    select x.id,y.id as yid,x.score,z.score as zscore from test x
    left join test y
        on x.id = y.id+1
        and x.id <= y.id+1
    left join test z
        on x.id = z.id+2
        and x.id >= z.id+1
    where x.id < 10
        and z.id < 10
)
select * from x limit 5
2022-01-20 23:58:19,203 [INFO] done


,id,yid,score,zscore
0,2,1,0.853854,0.764839
1,3,2,0.747254,0.897606
2,4,3,0.195149,0.853854
3,5,4,0.204596,0.747254
4,6,5,0.392279,0.195149


In [16]:
(
    pg.qry('test x')
    .select('x.cat,y.cat as bcat'
        ,'sum(x.score) bscore','sum(y.score) yscore','count(1) n')
    .join("test y","x.id = y.id+1")
    .where('x.id < 1000')
    .group_by('x.cat,y.cat')
    .having('count(1) > 50','sum(y.score) > 100')
    .order_by('cat desc','yscore desc')
    .run()
)

2022-01-20 23:59:36,281 [INFO] running:
select x.cat,y.cat as bcat,sum(x.score) bscore,sum(y.score) yscore,count(1) n
from test x
left join test y
    on x.id = y.id+1
where x.id < 1000
group by x.cat,y.cat
having count(1) > 50
    and sum(y.score) > 100
order by cat desc,yscore desc
2022-01-20 23:59:36,311 [INFO] done


,cat,bcat,bscore,yscore,n
0,train,test,126.746249,123.964186,248
1,train,train,122.177759,116.216551,245
2,test,train,130.094694,132.062596,247
3,test,test,114.607564,121.502911,259
